In [1]:
#Import dependenices
import csv
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import tensorflow as tf
import keras_tuner as kt
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
from sklearn.model_selection import train_test_split

# Split the Data into Training and Testing Sets

In [2]:
# Specify the path to your CSV file
loan_path = '../Resources/Clean_train_scaled.csv'

# Read the CSV file into a DataFrame
loan_df = pd.read_csv(loan_path)

# Display the DataFrame
loan_df.head(5)


,Current loan amount,Credit score,Annual income,Monthly debt,Years of credit history,Number of open accounts,Number of credit problems,Current credit balance,Maximum open credit,Long term,Short term,Home mortgage,Own home,Rent,Bankrupt
0,-0.403050,-0.248996,-0.195065,-1.088060,-0.139697,-1.023574,1.699378,-0.175312,-0.046075,0,1,1,0,0,1
1,-0.405859,-0.240861,-0.528623,-0.799685,-0.881886,-0.426898,-0.348918,-0.101568,-0.050396,0,1,0,1,0,0
2,-0.409913,4.212629,-0.445444,-0.176207,-0.125424,-1.023574,-0.348918,-0.209072,-0.067003,1,0,0,1,0,0
3,-0.400080,-0.270013,1.092393,0.011228,0.631038,-1.421359,-0.348918,0.372367,-0.026019,0,1,1,0,0,0
4,-0.409645,-0.228658,0.070678,1.696882,-0.610702,1.760918,-0.348918,0.981391,0.041627,0,1,1,0,0,1


In [3]:
loan_df.shape

(65072, 15)

In [4]:
# Remove target from features data
y = loan_df.Bankrupt.values
X = loan_df.drop(columns='Bankrupt').values


In [5]:
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)


# Create a Neural Network Model

In [6]:
# Create the Keras Sequential model
nn_model = tf.keras.models.Sequential()

# Add our first Dense layer, including the input layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="relu", input_dim=14))

# Add the output layer that uses a probability activation function
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the Sequential model
nn_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 15        
                                                                 
 dense_1 (Dense)             (None, 1)                 2         
                                                                 
Total params: 17 (68.00 Byte)
Trainable params: 17 (68.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [7]:
# Compile the Sequential model together and customise metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [8]:
# Fit the model to the training data
fit_model = nn_model.fit(X_train, y_train, epochs=50)

Epoch 1/50


1526/1526 [==============================] - 12s 6ms/step - loss: 0.4093 - accuracy: 0.8885
Epoch 2/50
1526/1526 [==============================] - 9s 6ms/step - loss: 0.3516 - accuracy: 0.8885
Epoch 3/50
1526/1526 [==============================] - 9s 6ms/step - loss: 0.3494 - accuracy: 0.8885
Epoch 4/50
1526/1526 [==============================] - 9s 6ms/step - loss: 0.3491 - accuracy: 0.8885
Epoch 5/50
1526/1526 [==============================] - 9s 6ms/step - loss: 0.3491 - accuracy: 0.8885
Epoch 6/50
1526/1526 [==============================] - 9s 6ms/step - loss: 0.3490 - accuracy: 0.8885
Epoch 7/50
1526/1526 [==============================] - 11s 7ms/step - loss: 0.3491 - accuracy: 0.8885
Epoch 8/50
1526/1526 [==============================] - 11s 7ms/step - loss: 0.3490 - accuracy: 0.8885
Epoch 9/50
1526/1526 [==============================] - 11s 7ms/step - loss: 0.3490 - accuracy: 0.8885
Epoch 10/50
1526/1526 [==============================] - 11s 7ms/step - loss

In [9]:
model_loss, model_accuracy = nn_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

509/509 - 3s - loss: 0.3502 - accuracy: 0.8885 - 3s/epoch - 6ms/step
Loss: 0.35024070739746094, Accuracy: 0.88849276304245


In [10]:
# Making predictions uisng the testing data
predictions =(nn_model.predict(X_test) > 0.5).astype("int32")
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score3 = accuracy_score(y_test, predictions)

509/509 [==============================] - 3s 5ms/step


In [11]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score3}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,14454,0
Actual 1,1814,0


Accuracy Score : 0.8884927464961888
Classification Report
              precision    recall  f1-score   support

           0       0.89      1.00      0.94     14454
           1       0.00      0.00      0.00      1814

    accuracy                           0.89     16268
   macro avg       0.44      0.50      0.47     16268
weighted avg       0.79      0.89      0.84     16268



C:\Users\wala2\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wala2\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wala2\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Auto Optimisation

In [12]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=14))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 10)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=5,
            max_value=50,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [13]:

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=30,
    hyperband_iterations=2) 

# Run the kerastuner search for best hyperparameters
tuner.search(X_train,y_train,epochs=30,validation_data=(X_test,y_test))

Trial 180 Complete [00h 07m 54s]
val_accuracy: 0.88849276304245

Best val_accuracy So Far: 0.88849276304245
Total elapsed time: 05h 25m 37s


In [14]:

# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 5,
 'num_layers': 2,
 'units_0': 50,
 'tuner/epochs': 2,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 3,
 'tuner/round': 0,
 'units_1': 5}

In [15]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

509/509 - 3s - loss: 0.3510 - accuracy: 0.8885 - 3s/epoch - 6ms/step
Loss: 0.35103029012680054, Accuracy: 0.88849276304245


In [16]:
# Making predictions uisng the testing data
predictions =(nn_model.predict(X_test) > 0.5).astype("int32")
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score3 = accuracy_score(y_test, predictions)

509/509 [==============================] - 3s 5ms/step


In [17]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score3}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,14454,0
Actual 1,1814,0


Accuracy Score : 0.8884927464961888
Classification Report
              precision    recall  f1-score   support

           0       0.89      1.00      0.94     14454
           1       0.00      0.00      0.00      1814

    accuracy                           0.89     16268
   macro avg       0.44      0.50      0.47     16268
weighted avg       0.79      0.89      0.84     16268



C:\Users\wala2\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wala2\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\wala2\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
